In [1]:
from ZMiner import *
from ZMinerD import *
from utils import *
from ZHist import *

In [2]:
data = load_data("synthetic_turbo_failure.xlsx", ['no','time','snap','date','status'] )

/data0/home/dsv/zele5930/projects/zhist_new/utils.py:431: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  a = data.drop(meta, axis=1)


In [3]:
data.head()

A                                                  ...     J             \
   A0   A1   A2    A3    A4    A5    A6   A7   A8  A9  ...    J5    J6   J7   
0   4   11   67   187   476   569   468  240   60  18  ...   472   293  130   
1  16   96  450  1154  1781  1771  1125  487  107  13  ...  1834  1125  420   
2  20  130  499  1121  1491  1106   471  135   24   3  ...  1171   611  194   
3   4   30  109   299   530   499   331  147   41  10  ...   477   295  124   
4   8   21   40    56    81    82    53   40   11   8  ...    87    43   15   

          no  time snap   date status  
   J8  J9                              
0  40  14  0  2100    0   2100      0  
1  76  15  0  7000    1   9100      0  
2  37   6  0  5000    2  14100      0  
3  30   5  0  2000    3  16100      0  
4   8   1  0   400    4  16500      0  

[5 rows x 229 columns]

In [4]:
# data, drop_cols, distance="chi2", minTrend = 300, 
# normal_term = 300, n_bins = 3, minimal_trend_gradient = 0,
# matrix_cols = [], remove_events = ['b']
# removeMatrix = False, removeTrend = True, isStandardized=False
z = ZHist(data, ['no','time','snap','date','status'], normal_term = 300, n_bins = 3, remove_events=['b'])

/data0/home/dsv/zele5930/projects/zhist_new/ZHist.py:147: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  self.data_simplified = data.drop(drop_cols, axis=1)


In [5]:
z.fit()

getWeightedAverage: 0.3333090841770172
interval creation started
current: A
current: B
current: C
current: D
current: E
current: F
current: G
current: H
current: I
current: J
interval creation is done: 245.78264617547393
interval separation started
interval separation is done: 0.009128373116254807


In [6]:
z.repair_intervals_removed

[[(('A', 'c'), 4400, 6500),
  (('A', 'a'), 8300, 0),
  (('A', 'c'), 0, 3300),
  (('A', 'a'), 8400, 9900),
  (('A', 'c'), 11700, 13200),
  (('A', 'c'), 13500, 16200),
  (('A', 'a'), 21000, 26400),
  (('A', 'c'), 26700, 27600),
  (('A', 'a'), 28800, 31500),
  (('A', 'a'), 32400, 34500),
  (('A', 'a'), 35100, 37200),
  (('A', 'c'), 37200, 37600),
  (('A', 'a'), 37600, 38800),
  (('A', 'c'), 39100, 40600),
  (('A', 'a'), 40900, 42700),
  (('A', 'c'), 43000, 47500),
  (('A', 'a'), 48100, 49300),
  (('A', 'c'), 49900, 50700),
  (('B', 'c'), 10100, 0),
  (('C', 'c'), 10100, 0),
  (('D', 'a'), 4100, 7700),
  (('D', 'a'), 10400, 14300),
  (('D', 'a'), 0, 1500),
  (('D', 'a'), 8700, 9900),
  (('D', 'c'), 11700, 13500),
  (('D', 'a'), 13800, 15600),
  (('D', 'c'), 16800, 28800),
  (('D', 'a'), 28800, 30300),
  (('D', 'c'), 30300, 31800),
  (('D', 'a'), 32100, 33000),
  (('D', 'c'), 33300, 35100),
  (('D', 'a'), 36000, 36600),
  (('D', 'c'), 37200, 38800),
  (('D', 'a'), 38800, 40000),
  (('D', 'c

In [7]:
database = Database(z.repair_intervals_removed)
# minSup, epsilon (= 0), gap, timeout, level
constraints = makeConstraints([0.1, 0, 1000, 2000000000, 3], z.repair_intervals_removed)
algorithm = ZMiner(database, constraints, forgettable=True)
# algorithm.constraints["timeoutseconds"] = 2000000000
count, freq, timedelta, timeout, FL_repair = algorithm.ZMiner()

########## Z-MINER ##########
1-1. MINIMUM SUPPORT: 10.0
1-2. EPSILON CONSTRAINT: 0.0
1-3. GAP CONSTRAINT: 1000.0
1-4. TIMEOUT: 2000000000
1-5. LEVEL: 3.0
2. NUMBER OF E-SEQUENCES: 100
3. TOTAL COMPARISON COUNTS: 731625
4. TOTAL FREQUENT ARRANGEMENTS: 7668
5. TOTAL TIME CONSUMED: 6.771270743999992


In [8]:
database2 = Database(z.normal_intervals_removed)
constraints2 = makeConstraints([0.1, 0, 1000, 200000000, 3], z.normal_intervals_removed)
algorithm2 = ZMinerD(database2, constraints2, FL_repair, forgettable=True)
# algorithm2.constraints["timeoutseconds"] = 200000000
count2, freq2, tdelta2, timeout2, FL_normal = algorithm2.ZMiner()

########## Z-MINER ##########
1-1. MINIMUM SUPPORT: 90.0
1-2. EPSILON CONSTRAINT: 0.0
1-3. GAP CONSTRAINT: 1000.0
1-4. TIMEOUT: 200000000
2. NUMBER OF E-SEQUENCES: 900
3. TOTAL COMPARISON COUNTS: 1750200
4. TOTAL FREQUENT ARRANGEMENTS: 725
5. TOTAL TIME CONSUMED: 18.59606928000005


In [22]:
def exportDisprop(dataname, FL1, FL2, n1, n2, constraints):            
    filename = (
        "Disprop"
        + "_"
        + dataname
        + "_"
        + str(constraints["minSupPercent"])
        + "_"
        + str(constraints["epsilon"])
        + "_"
        + str(constraints["gap"])
        + "_"
        + str(constraints["timeoutseconds"])
        + ".csv"
    )
    F = []
    for k in FL1:
        for E in FL1[k]:
            for R in FL1[k][E]:
                if type(FL1[k][E][R]) != int:
                    length1 = len(FL1[k][E][R])
                else:
                    length1 = FL1[k][E][R]
                if k not in FL2 or E not in FL2[k] or R not in FL2[k][E]:
                    F.append({"events": E, "relations": R, "freq1": length1, "freq2": 0,
                    "relSup1": (length1+1)/(n1+1), "relSup2": 0,
                    "disprop": (((length1)+1)/(n1+1))/(1/(n2+1))})
                else:
                    if type(FL2[k][E][R]) != int:
                        length2 = len(FL2[k][E][R])
                    else:
                        length2 = FL2[k][E][R]
                    # length2 = FL2[k][E][R]
                    F.append({"events": E, "relations": R, "freq1": length1, "freq2": length2,
                    "relSup1": (length1+1)/(n1+1), "relSup2": (length2+1)/(n2+1),
                    "disprop": ((length1+1)/(n1+1))/((length2+1)/(n2+1))})
    
    with open(filename, "w") as output_file:
        dict_writer = csv.DictWriter(output_file, ["events", "relations", "freq1", "freq2", "relSup1", "relSup2", "disprop"])
        dict_writer.writeheader()
        dict_writer.writerows(F)
    return filename

In [23]:
exportDisprop("result", FL_repair, FL_normal, 100, 900, constraints2)

'Disprop_result_0.1_0.0_1000.0_200000000.csv'